# Train best Ridge model (for visualisation of coefficients)


## 1. Preprocessing

In the first step, you will preprocess the CONN Matlab files to a analysis ready dataset.

Here is an overview on the parameters for the preprocessing pipeline. Parameters marked with a (*) are optional.


+    *matlab_dir*: path to matlab files
+    *excel_path*: path to excel list
+    *save_file*: If false return as pd dataframe
+    *preprocessing_type*: conn for connectivity matrix, "aggregation" for aggregated conn matrix, "graph" for graph matrices
+    *write_dir**: path where to write the dataset to if save_file = True
+    *network**: Yeo7 or Yeo17 network (only applicable if preprocessing_type = aggregation)
+    *statistic**: Summary statistic to be applied (only applicable if preprocessing_type = aggregation)
+    *upper**: boolean whether only upper diagonal elements of connecivity matrices should be used
+    *split_size**: the size of the train dataset (default .8)
+    *seed**: pass an int for reproducibility purposes (default 42)
+    *file_format**: Pass "h5" for further modelling in python or "csv" for R (default "csv")

In [ ]:
import os
import pandas as pd
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github')

Mounted at /content/drive


In [ ]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files

In [ ]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'conn' 

In [ ]:
df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = 'conn')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [36]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_14,1_15,1_16,1_17,1_18,1_19,1_20,1_21,1_22,1_23,1_24,1_25,1_26,1_27,1_28,1_29,1_30,...,237_243,237_244,237_245,237_246,238_239,238_240,238_241,238_242,238_243,238_244,238_245,238_246,239_240,239_241,239_242,239_243,239_244,239_245,239_246,240_241,240_242,240_243,240_244,240_245,240_246,241_242,241_243,241_244,241_245,241_246,242_243,242_244,242_245,242_246,243_244,243_245,243_246,244_245,244_246,245_246
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,3.0,0.75266,0.231123,0.149763,0.450447,0.185762,0.599392,0.156499,0.782779,0.427059,0.327918,0.181461,0.017913,-0.112979,0.183447,0.120085,0.279274,0.116219,0.322515,0.155501,-0.090248,-0.125024,0.1874,-0.05036,0.586258,0.491451,-0.030206,-0.222713,0.031067,-0.160859,...,0.849411,0.71605,-0.025114,0.291594,-0.051196,0.082976,-0.24298,0.118436,0.250335,0.286292,-0.198465,0.158462,0.279858,0.002769,0.119833,0.023414,0.007263,0.150606,0.057617,-0.023502,0.552398,-0.001833,0.141841,0.069829,0.435568,0.243923,0.134239,0.033118,0.132755,-0.016676,0.209853,0.166607,0.056147,0.159351,0.721864,0.107519,0.329301,0.055294,0.39387,0.269579
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,0.700115,0.815505,0.480916,0.274557,0.048538,0.719735,0.46748,0.521357,0.449369,0.952166,0.511867,0.49259,-0.023851,0.647918,0.305384,0.749082,0.254135,0.586528,0.076689,0.323022,0.080136,0.8353,0.366546,0.774213,0.567104,0.323971,0.095188,0.100919,0.375862,...,0.57138,0.541166,-0.276125,-0.038445,-0.105363,0.090075,-0.160635,0.673594,0.663931,0.624048,-0.128801,-0.008332,0.532339,0.520765,-0.029643,0.181075,-0.005707,0.577066,0.498839,0.251391,0.396573,0.401372,0.27947,0.280546,0.154698,0.095783,0.095389,-0.211978,0.388244,0.257077,0.795011,0.624784,-0.115689,0.055597,0.684795,0.212303,0.193802,-0.117644,-0.092837,0.621967
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,1.0,1.430655,0.958171,0.51861,0.591034,0.227465,0.787522,0.776009,0.798374,0.999554,0.954094,0.837725,0.197717,0.270467,0.48662,0.380078,0.606075,0.605735,0.444788,0.29387,0.379651,0.481484,0.318099,0.13934,0.727726,0.699307,-0.207808,-0.32161,0.592996,0.621801,...,0.35924,0.56932,0.145873,0.152402,-0.075421,0.039472,-0.0109,0.147892,0.19681,0.327893,-0.064144,0.058777,0.327169,0.295942,0.325517,0.226479,-0.045583,0.194034,0.246963,0.798343,0.968418,0.716178,0.343208,-0.034113,0.125901,0.587815,0.670493,0.249516,-0.032901,0.216782,0.655094,0.547549,-0.256346,0.079905,0.504077,0.031435,0.180466,0.095308,-0.062235,0.210861
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,16.0,0.749298,0.478309,-0.234489,0.26929,-0.167393,0.260917,-0.001153,0.523586,0.517067,0.572653,0.294286,0.126939,0.04613,0.633516,0.096745,-0.006272,-0.177413,-0.326773,-0.325359,0.128038,-0.173338,-0.000516,-0.502287,0.154206,-0.502589,0.234381,-0.021147,0.477312,-0.098342,...,0.708086,0.56764,0.086452,0.058225,0.030889,0.127198,0.227308,0.334972,0.223136,0.270046,-0.001241,0.070125,0.114546,0.094386,-0.188919,0.164224,0.171668,0.371653,0.377588,0.560254,0.773189,0.491207,0.416849,0.044909,0.090991,0.551999,0.346686,0.257181,-0.148415,0.013386,0.406717,0.40963,-0.178134,-0.017138,0.84083,0.163739,0.233597,0.164371,0.209712,0.437848
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,15.0,0.349147,0.182783,-0.157015,-0.006974,-0.209892,0.224349,0.037267,0.242256,0.208523,0.418078,0.181657,-0.036354,-0.291805,0.282012,0.152677,0.362262,0.235874,0.38177,-0.013424,-0.041562,0.020208,0.335079,0.113339,0.448809,0.13363,0.051833,-0.072217,0.359955,0.220076,...,0.349013,0.624345,0.241957,0.255162,0.363485,0.251562,0.458895,0.275598,0.40057,0.341686,0.360847,0.184512,0.417218,0.408215,0.198807,0.394086,0.225733,0.489461,0.653907,0.477328,0.619809,0.560182,0.380952,0.235424,0.099328,0.474011,0.653559,0.311723,0.154802,0.115935,0.526126,0.403969,0.071003,-0.118103,0.702142,-0.014435,0.048391,0.054875,0.001036,0.657731
...,...,...,...,...,...,.

## 2. Modelling

In the second step, you can decide between running the new input files on a pretrained model or train a new model

### 2.1  Data preparation
Preparation of the data for modelling. Creates the target variable, drops unnecessary columns, performs a train/test split (if wanted). \\
The user has to specify:
- *classification*: is it a classification task (True) or a regression task (False)
- *columns_drop*: which variables shoulnd't be used for modelling
- *target*: what is the name of the target variable
- *y_0, y_1* (only relevant for classification task): which values of the target variable are 0, which are 1
- *train_size*: size of the training data
- *seed*: a seed to ensure reproducibility of train/test split
- split: should a train/test split be performed or not? 

In [ ]:
from src.preprocessing.data_preparation import prepare_data

In [ ]:
classification = True
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "MEM_score", "Apoe", "IDs"]
target = "prmdiag"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [ ]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,1_2,1_3,1_4,1_5,1_6,1_7,1_8,1_9,1_10,1_11,1_12,1_13,1_14,1_15,1_16,1_17,1_18,1_19,1_20,1_21,1_22,1_23,1_24,1_25,1_26,1_27,1_28,1_29,1_30,1_31,1_32,1_33,1_34,1_35,1_36,1_37,...,237_243,237_244,237_245,237_246,238_239,238_240,238_241,238_242,238_243,238_244,238_245,238_246,239_240,239_241,239_242,239_243,239_244,239_245,239_246,240_241,240_242,240_243,240_244,240_245,240_246,241_242,241_243,241_244,241_245,241_246,242_243,242_244,242_245,242_246,243_244,243_245,243_246,244_245,244_246,245_246
30,0.0,1.774171,-1.149848,-0.727818,-0.175015,-0.607746,0.710297,-0.565162,-0.019367,0.622931,0.670039,0.761765,0.907808,-0.714585,-0.413560,-1.142146,0.591749,-1.066501,-0.205261,0.105115,0.669763,-0.627350,0.649609,-0.710452,-0.323599,-0.777914,-0.330134,-0.174241,0.232365,0.011136,0.770357,-0.369539,-0.001101,-0.291920,0.001114,0.131853,-0.407839,0.314248,0.681717,-0.704789,...,0.642271,0.290669,-1.138242,-1.752161,-0.060423,-2.106688,-0.885030,-1.327712,0.439793,0.555164,-0.864755,-1.013066,-0.032244,-0.224239,0.081408,0.442141,-0.472185,-0.349435,-0.519537,-0.843149,0.014823,0.380482,-0.216648,1.147353,0.864599,-1.394554,-0.785950,-1.374258,1.807872,1.116819,-0.538365,0.057557,0.172242,-0.438061,0.277015,-0.302949,-0.270495,-1.018721,-0.767116,1.025743
792,1.0,1.106080,-1.149848,2.001953,-1.643241,-0.946779,-1.479507,-0.043356,-1.016759,-0.825907,-1.832529,0.017217,-0.835740,-0.134362,-0.432902,-0.640025,-0.511792,-0.765991,-0.857584,0.172187,-1.885116,-0.339783,-1.139582,-1.403149,-1.279953,-0.234403,-0.956491,0.699611,-1.540705,-0.928739,-0.663811,-0.697963,-1.410021,-1.376664,-0.900358,1.726088,-1.684212,-0.442203,-2.294344,0.514593,...,-0.382461,-0.165517,-0.061162,0.141713,-0.665515,-0.987417,-0.869834,-1.263755,-1.339798,-1.558125,-0.485576,-1.290011,-0.170550,0.402297,-0.628019,0.352747,1.341268,-0.255718,-0.011129,-1.781210,-0.767330,0.108065,-0.124723,0.143002,0.711198,-0.207728,-0.480993,0.587440,-0.924691,-0.741742,-0.270957,-0.028714,-1.180537,-1.320330,0.164311,-0.872465,0.110509,0.042462,-0.057570,-0.155071
497,0.0,0.103945,0.469414,-0.386597,-1.128745,0.723342,-0.479988,0.868291,-0.708498,0.865226,0.582073,0.838496,0.301396,-0.189252,-0.383881,1.445869,-0.050086,0.215154,-0.457827,0.696638,-0.882751,-1.684970,-1.759451,-0.859228,-1.255707,-0.425754,-1.590043,0.547453,-1.651432,0.016212,-2.005010,-0.997738,-0.777929,-0.104273,-1.243611,-0.548417,-1.834410,0.852728,0.258680,0.285531,...,0.461139,0.612500,0.648193,1.470875,-0.063846,-0.177801,-0.637208,-1.060784,-0.288524,-1.007307,-0.475227,-0.237566,-0.527491,0.864047,-0.032899,0.366425,-0.682234,-0.325940,-0.134205,0.777965,0.893269,1.376280,1.833457,-0.298480,0.767538,0.237919,0.307264,1.634669,0.286409,1.065707,-0.105832,0.998391,0.317190,0.749562,-0.183566,1.071840,0.845802,0.980418,0.672400,0.365327
594,1.0,1.440126,-1.149848,1.660732,-0.002840,-1.083258,-1.258788,0.816870,-0.192023,-1.223150,-0.865689,-0.369847,-0.337219,-1.061589,-1.027766,-1.058457,-1.188337,0.445268,-0.550863,-1.223637,-1.771768,-0.445121,0.242214,-0.715170,-1.702007,-1.119130,-0.795495,-0.663670,-1.381225,-0.431259,-0.452878,2.163063,-1.179705,-0.521198,-1.723510,-0.258259,0.503473,-0.902302,-1.349328,1.423196,...,1.009767,1.293996,0.570681,1.677660,0.573096,-0.380398,-0.591804,-0.887186,-0.929303,0.693277,-0.032684,0.780622,1.141231,1.317205,0.723131,0.232938,-0.009904,1.697534,1.279773,1.451142,1.330102,1.457354,0.358575,1.455573,1.173627,0.640307,1.339681,0.335741,1.337385,0.387070,1.215576,0.361916,0.621995,1.254900,0.113544,0.647532,0.495780,0.586744,0.953708,1.088182
712,0.0,0.772035,0.469414,-0.727818,-2.519814,-0.848912,-0.783948,-0.313696,-0.713626,-1.304876,-1.561134,-0.634300,-1.540037,-1.048967,-0.534788,-0.249076,-0.655689,-0.519040,-1.256621,-0.041519,-0.408635,-0.385107,-0.102291,0.000467,-1.067582,0.693141,0.146569,0.242165,-0.785957,0.887896,0.386920,-0.690760,-2.086559,-0.682899,-1.429059,0.074374,-0.356426,-1.281304,-0.713868,0.610868,...,-1.213876,-1.765250

## Train Model

In [ ]:
from src.models.pipeline_elastic_net import model_elastic_net

In [34]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, classification = True, 
                          n_alphas_logreg = 10, 
                          l1_ratios_logreg=[1.0], 
                          cv_logreg = 3)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter

In [35]:
from sklearn.metrics import accuracy_score

predictions = model.predict(Xtest)
accuracy_score(ytest, predictions)

0.5319148936170213

In [27]:
model.l1_ratio_

array([1.])

In [28]:
model.C_

array([0.04641589])

In [31]:
ytrain.value_counts()

1.0    200
0.0    176
Name: y, dtype: int64

In [32]:
ytest.value_counts()

1.0    52
0.0    42
Name: y, dtype: int64

In [33]:
52/(52+42)

0.5531914893617021

# Save Model

In [ ]:
import pickle

In [ ]:
filename ='elastic_net_class_ridge.sav'
pickle.dump(model, open(filename, 'wb'))

## Visualisation Coefficients

In [ ]:
from src.visualization.viz_utils import plot_coef_elastic_net

ModuleNotFoundError: ignored

In [ ]:
plot_coef_elastic_net(model)

NameError: ignored